from pyspark import SparkConf, SparkContext
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
import gc
sc = SparkContext(master = 'local')
spark = SparkSession.builder.appName('pySpark proj3_test').getOrCreate()

In [1]:
import pandas as pd
import numpy as np
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import Tokenizer, RegexTokenizer, HashingTF
from pyspark.sql import SparkSession

In [2]:
# spark setup
from pyspark import SparkContext, SparkConf
sc = SparkContext(master = 'local')
spark = SparkSession.builder.appName('pySpark word-count').config('spark.some.config.option', 'some-value')             .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/14 17:22:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# data import
data = spark.read.format("csv").option("header",True).load("/Users/iris/spam_data.csv") \
    .withColumnRenamed("Category", "label") \
    .withColumnRenamed("Message", "text")
data_df = data.toDF("label", "text")

In [4]:
data_df.show(3)
print(data_df.count(), data_df.columns)
type(data_df)

+-----+--------------------+
|label|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
+-----+--------------------+
only showing top 3 rows

5574 ['label', 'text']


pyspark.sql.dataframe.DataFrame

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
# replace label spam = 0 and ham = 1
data_df = data_df.withColumn('label', regexp_replace('label', 'spam', '0'))
data_df = data_df.withColumn('label', regexp_replace('label', 'ham', '1'))

# cast those string categories as int dtype
data_df = data_df.withColumn('label', data_df.label.cast(IntegerType()))

print(data_df.printSchema())

root
 |-- label: integer (nullable = true)
 |-- text: string (nullable = true)

None


In [45]:
# # pre-processing
# post_data = Tokenizer(inputCol='text', outputCol='words').transform(data_df)

# #test_post_data = RegexTokenizer(inputCol='text', outputCol='token', pattern='\\W+')

# #countTokens = udf(lambda x: len(x), IntegerType())
# post_data.show(4)
# post_data.columns
# from pyspark.sql.functions import col 
# post_data.select('label', 'text', 'words').withColumn('token', countTokens(col('words'))).show(5)

In [6]:
data_df.show(4)


+-----+--------------------+
|label|                text|
+-----+--------------------+
|    1|Go until jurong p...|
|    1|Ok lar... Joking ...|
|    0|Free entry in 2 a...|
|    1|U dun say so earl...|
+-----+--------------------+
only showing top 4 rows



In [7]:
#data_df.filter(data_df.label.isNull()).show(5)

+-----+--------------------+
|label|                text|
+-----+--------------------+
| null| give me a call i...|
| null| DEVIOUSBITCH.ANYWAY|
+-----+--------------------+



In [10]:
data_df = data_df.dropna()
print(data_df.count(), data_df.columns)

5572 ['label', 'text']


In [11]:

training_set, testing_set = data_df.randomSplit([0.8, 0.2])

#tokenise
# 1. clean data and tokenize sentences using RegexTokenizer
tokenizer = Tokenizer(inputCol='text', outputCol='words')

# 2. hashTF the data
hasherTF = HashingTF(inputCol='words', outputCol='features')

# 3. classifier
lr = LogisticRegression(maxIter=20, regParam=0.01)

In [16]:
# pipeline
pipeline = Pipeline(stages = [tokenizer, hasherTF, lr])
model_fit = pipeline.fit(training_set)

In [17]:
# predictions
training_predictions = model_fit.transform(training_set)
test_predictions = model_fit.transform(testing_set)